In [7]:
import pandas as pd
import networkx as nx
from pyvis.network import Network

In [8]:
docs_df = pd.read_csv("../data/law_analyzer_new99_generated_topics.csv")
docs_df

,nama_file,nomor_peraturan,judul,tahun,nomor,konsideran,heading,definition_term,token_count,generated_topics
0,01~pmk.010~2011.json,PMK 01/PMK.010/2011,Perubahan atas Peraturan Menteri Keuangan Nomo...,2011,1,a. bahwa dalam rangka memperoleh perhitungan ...,[],NaN,258,Format laporan; Tarif premi; Pendapatan; Profi...
1,02~pmk.05~2011.json,PMK 02/PMK.05/2011,Pedoman Akuntansi Dan Pelaporan Aset Berupa Ba...,2011,2,a. bahwa berdasarkan Peraturan Menteri Keuang...,"['Pengertian', 'Ruang Lingkup', 'Sistem Akunta...","['Kontraktor Kontrak Kerja Sama,', 'Barang yan...",397,Pedoman Akuntansi; Pelaporan Aset; Barang Mili...
2,04~pmk.02~2014.json,PMK 04/PMK.02/2014,Perubahan Kedua atas Peraturan Menteri Keuanga...,2014,4,a. bahwa dalam rangka mendukung agar kondisi ...,[],NaN,341,Perimbangan kekayaan dan kewajiban; Badan peny...
3,04~pmk.06~2010.json,PMK 04/PMK.06/2010,Penilai Internal di Lingkungan direktorat Jend...,2010,4,a. bahwa dalam rangka pengelolaan kekayaan ne...,"['Umum', 'Syarat Pengangkatan', 'Usulan Pengan...","['Menteri', 'Direktorat Jenderal', 'Kantor Pus...",205,Penilaian; Pengelolaan; Piutang negara; Pelaya...
4,04~pmk.07~2011.json,PMK 4/PMK.07/2011,Tata Cara Penyampaian Informasi Keuangan Daerah,2011,4,a. bahwa untuk melaksanakan Peraturan Pemerin...,[],"['Pemerintah Daerah', 'Daerah otonom,', 'Daera...",244,Peraturan Pemerintah; Sistem Informasi Keuanga...
...,...,...,...,...,...,...,...,...,...,...
95,135~pmk.08~2019.json,PMK 135 /PMK.08/2019,Perubahan atas Peraturan Menteri Keuangan Nomo...,2019,135,a. bahwa untuk melaksanakan ketentuan Pasal 7...,[],"['Anggaran Kewajiban Penjaminan Pemerintah', '...",419,Perubahan; Pelaksanaan; Pemberian Jaminan Peme...
96,136~pmk.03~2012.json,PMK 136/PMK.03/2012,Perubahan atas Peraturanmenteri Keuangan Nomor...,2012,136,a. Bahwa ketentuan mengenai penunjukan pemung...,[],NaN,379,Penunjukan pemungut; Kemudahan bagi BUMN; Peny...
97,137~pmk.01~2016.json,PMK 137/PMK.01/2016,Aktuaris,2016,137,a. bahwa aktuaris dibutuhkan dalam pengembang...,"['Izin Aktuaris Publik', 'Pengunduran Diri dan...","['Ajun Aktuaris Beregister', 'Aktuaris Beregis...",292,Pengembangan bidang ekonomi; Industri perasura...
98,137~pmk.01~2019.json,PMK 137 /PMK.01/2019,Organisasi Dan Tata Kerja Badan Pengelola Dana...,2019,137,a. bahwa untuk melaksanakan ketentuan Pasal 8...,[],['(1) Badan Pengelola Dana Lingkungan Hidup'],300,Pembentukan unit organisasi; Pengelolaan dana ...


In [9]:
docs_df = pd.read_csv("../data/law_analyzer_new99_generated_topics.csv")

# select only nomor_peraturan and generated_topics columns
docs_df = docs_df[['nomor_peraturan', 'generated_topics']]

# transform each value in the generated_topics column to a list
docs_df['generated_topics'] = docs_df['generated_topics'].apply(lambda x: x.split("; "))

# remove dot (.) character in every last value of the generated_topics column
docs_df['generated_topics'] = docs_df['generated_topics'].apply(lambda x: [topic[:-1] if topic[-1] == '.' else topic for topic in x])

# lower case all topics
docs_df['generated_topics'] = docs_df['generated_topics'].apply(lambda x: [topic.lower() for topic in x])

docs_df

,nomor_peraturan,generated_topics
0,PMK 01/PMK.010/2011,"[format laporan, tarif premi, pendapatan, prof..."
1,PMK 02/PMK.05/2011,"[pedoman akuntansi, pelaporan aset, barang mil..."
2,PMK 04/PMK.02/2014,"[perimbangan kekayaan dan kewajiban, badan pen..."
3,PMK 04/PMK.06/2010,"[penilaian, pengelolaan, piutang negara, pelay..."
4,PMK 4/PMK.07/2011,"[peraturan pemerintah, sistem informasi keuang..."
...,...,...
95,PMK 135 /PMK.08/2019,"[perubahan, pelaksanaan, pemberian jaminan pem..."
96,PMK 136/PMK.03/2012,"[penunjukan pemungut, kemudahan bagi bumn, pen..."
97,PMK 137/PMK.01/2016,"[pengembangan bidang ekonomi, industri perasur..."
98,PMK 137 /PMK.01/2019,"[pembentukan unit organisasi, pengelolaan dana..."


In [10]:
# Membuat graph kosong
G = nx.Graph()

# Menambahkan nodes dan edges berdasarkan topik dan nomor peraturan
for idx, row in docs_df.iterrows():
    nomor_peraturan = row['nomor_peraturan']
    topik_peraturan = row['generated_topics']
    
    # Menambahkan node untuk nomor peraturan
    G.add_node(nomor_peraturan, label=nomor_peraturan, color='blue')
    
    # Menambahkan node untuk setiap topik dan edge dari topik ke nomor peraturan
    for topik in topik_peraturan:
        G.add_node(topik, label=topik, color='green')
        G.add_edge(nomor_peraturan, topik)

# Debugging: Print nodes and edges
print("Nodes:", G.nodes(data=True))
print("Edges:", G.edges(data=True))

Nodes: [('PMK 01/PMK.010/2011', {'label': 'PMK 01/PMK.010/2011', 'color': 'blue'}), ('format laporan', {'label': 'format laporan', 'color': 'green'}), ('tarif premi', {'label': 'tarif premi', 'color': 'green'}), ('pendapatan', {'label': 'pendapatan', 'color': 'green'}), ('profil risiko', {'label': 'profil risiko', 'color': 'green'}), ('kerugian', {'label': 'kerugian', 'color': 'green'}), ('data biaya', {'label': 'data biaya', 'color': 'green'}), ('administrasi', {'label': 'administrasi', 'color': 'green'}), ('biaya umum', {'label': 'biaya umum', 'color': 'green'}), ('pemasaran', {'label': 'pemasaran', 'color': 'green'}), ('asuransi kendaraan bermotor', {'label': 'asuransi kendaraan bermotor', 'color': 'green'}), ('perubahan', {'label': 'perubahan', 'color': 'green'}), ('peraturan menteri keuangan', {'label': 'peraturan menteri keuangan', 'color': 'green'}), ('PMK 02/PMK.05/2011', {'label': 'PMK 02/PMK.05/2011', 'color': 'blue'}), ('pedoman akuntansi', {'label': 'pedoman akuntansi', 'co

In [11]:
# Membuat visualisasi interaktif dengan pyvis
net = Network(notebook=True)
net.from_nx(G)

In [13]:
net.show("../data/law_analyzer_new99_generated_topics.html")

../data/law_analyzer_new99_generated_topics.html
